In [5]:
import pandas as pd

In [4]:
!pip install pandas

In [27]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
#import pyLDAvis.gensim

#https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
!pip install -U spacy

  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)


In [23]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [25]:
!pip install pyLDAvis

  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached funcy-1.17-py2.py3-none-any.whl (33 kB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=eb5c6e1dabeb169b83f16970bb2b9ca2fc81085f4ac95720f089529d76d50681
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\57\a4\86\d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=ccaf3b12e45c51a8e4f742bc95676d3afc51f38cb1a0a945584ee7df5cfa93a

In [216]:
data = pd.read_csv('Sanctions/chord/highlighted_sentiments_survey.csv', error_bad_lines=False);
data_text = data[['CV2']]
documents = data_text

D:\Anaconda\Anaconda_installation\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [218]:
print(len(documents))
print(documents[:5])

1150
                                                 CV2
0  Store i was working in had to close due the re...
1  I could not find a job now because everything ...
2  Lectures are cancelled and exams postponed, it...
3  During the last month I am not working in a bi...
4  I have not been able to go to work which made ...


In [219]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [220]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [221]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})



,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


 # Data Pre-processing
   - Tokenization: Split the text into sentences and the sentences into words. 
   - Lowercase the words and remove punctuation.
   - Words that have fewer than 3 characters are removed.
   -  All stopwords are removed.
   - Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
   -  Words are stemmed — words are reduced to their root form.

In [222]:


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result



# whithout stemming only lemmatization then run the following code

# whithout stemming

```python
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
```

In [223]:
processed_docs = documents['CV2'].fillna('').astype(str).map(preprocess)
# fillna('').astype(str)  --> helps to ignore the empty rows (Nan) in CV2 column.


# processed_docs = documents['CV2'].map(preprocess)

In [224]:
 documents['CV2'][200]

'I am not working so that means that I am not earning money'

In [225]:
 documents['CV2'][600]

'Mostly the only real change is that I am working from home and not seeing anyone in person -- I am taking more calls and reaching out to friends via zoom for socialization, and going to "virtual" events at times. I also am making an effort to do things that "fill me up", like hobbies and exercise and reading. I am cooking all my food at home, and freezing leftovers and produce to avoid having to go to the stores, which are out of many items.'

In [226]:
processed_docs[:10]

0                       [store, work, close, restrict]
1         [close, student, univers, close, home, shut]
2    [lectur, cancel, exam, postpon, difficult, pla...
3    [month, work, compani, anymor, current, situat...
4    [abl, work, negat, impact, financi, situat, un...
5    [easi, order, groceri, shop, deliv, groceri, f...
6                         [famili, stay, home, stress]
7                                    [stay, quaranten]
8      [hasn, affect, home, offic, wife, miss, honest]
9                                         [hour, work]
Name: CV2, dtype: object

# Bag of words on the CV2 column

Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [227]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [228]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 close
1 restrict
2 store
3 work
4 home
5 shut
6 student
7 univers
8 cancel
9 difficult
10 exam


# Gensim filter_extremes
## Filter out tokens that appear in

- less than 15 documents (absolute number) or
- more than 0.5 documents (fraction of total corpus size, not absolute number). 
- after the above two steps, keep only the first 100000 most frequent tokens.

In [229]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear.
Save this to ‘bow_corpus’, then check our selected document earlier.

In [230]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[200]

[(3, 1), (53, 1), (120, 1)]

In [231]:
bow_corpus[200]

[(3, 1), (53, 1), (120, 1)]

In [232]:
 documents['CV2'][200]

'I am not working so that means that I am not earning money'

In [233]:
doc_sample =  documents['CV2'][200]

In [234]:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['I', 'am', 'not', 'working', 'so', 'that', 'means', 'that', 'I', 'am', 'not', 'earning', 'money']


 tokenized and lemmatized document: 
['work', 'mean', 'earn', 'money']


# Select a document to preview after preprocessing.
## Preview Bag Of Words for our sample preprocessed document.

In [235]:
bow_doc_200 = bow_corpus[200]

for i in range(len(bow_doc_200)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_200[i][0], 
                                                     dictionary[bow_doc_200[i][0]], 
                                                     bow_doc_200[i][1]))



Word 3 ("work") appears 1 time.
Word 53 ("money") appears 1 time.
Word 120 ("mean") appears 1 time.


In [236]:
bow_doc_600 = bow_corpus[600]

for i in range(len(bow_doc_600)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_600[i][0], 
                                                     dictionary[bow_doc_600[i][0]], 
                                                     bow_doc_600[i][1]))


Word 2 ("store") appears 1 time.
Word 3 ("work") appears 1 time.
Word 4 ("home") appears 2 time.
Word 12 ("chang") appears 1 time.
Word 22 ("thing") appears 1 time.
Word 30 ("social") appears 1 time.
Word 34 ("like") appears 1 time.
Word 55 ("time") appears 1 time.
Word 62 ("person") appears 1 time.
Word 66 ("friend") appears 1 time.
Word 68 ("have") appears 1 time.
Word 74 ("go") appears 1 time.
Word 78 ("take") appears 1 time.
Word 140 ("food") appears 1 time.
Word 153 ("make") appears 1 time.
Word 191 ("exercis") appears 1 time.
Word 194 ("item") appears 1 time.


#  TF-IDF
TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.

This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

So, words that are common in every document, such as `this, what, and if`, rank low even though they may appear many times, since they don’t mean much to that document in particular.

However, if the word "Bug" appears many times in a document, while not appearing many times in others, it probably means that it’s very relevant.

- Create `tf-idf model` object using `models.TfidfModel` on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [237]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [238]:
corpus_tfidf = tfidf[bow_corpus]

In [239]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4744877646555355),
 (1, 0.6553798865530638),
 (2, 0.5665436263439183),
 (3, 0.15609895881449373)]


TF-IDF for a word in a document is calculated by multiplying two different metrics:

- The `term frequency` TF of a word in a document. There are several ways of calculating this frequency, with the simplest being a raw count of instances a word appears in a document. Then, there are ways to adjust the frequency, by length of a document, or by the raw frequency of the most frequent word in a document.
- The `inverse document frequency` IDF  of the word across a set of documents. This means, how common or rare a word is in the entire document set. The closer it is to 0, the more common a word is. This metric can be calculated by taking the total number of documents, dividing it by the number of documents that contain a word, and calculating the logarithm.
    - So, if the word is very common and appears in many documents, this number will approach 0. Otherwise, it will approach 1.

# Running LDA using Bag of Words
Train our LDA model using `gensim.models.LdaMulticore` and save it to ‘lda_model’

In [240]:
num_topics = 10
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics= num_topics, id2word=dictionary, passes=2, workers=2)

# Print the Keyword in the 10 topics

For each topic, we will explore the words occuring in that topic and its relative weight.

In [241]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))



Topic: 0 
Words: 0.082*"work" + 0.050*"home" + 0.048*"affect" + 0.020*"go" + 0.019*"social" + 0.018*"health" + 0.017*"life" + 0.017*"school" + 0.016*"covid" + 0.014*"impact"
Topic: 1 
Words: 0.079*"home" + 0.069*"work" + 0.055*"time" + 0.023*"week" + 0.021*"groceri" + 0.019*"person" + 0.018*"virus" + 0.018*"hous" + 0.018*"stay" + 0.015*"coronavirus"
Topic: 2 
Words: 0.043*"famili" + 0.043*"work" + 0.039*"home" + 0.034*"feel" + 0.026*"stay" + 0.026*"affect" + 0.022*"hous" + 0.019*"friend" + 0.017*"peopl" + 0.016*"get"
Topic: 3 
Words: 0.086*"home" + 0.057*"work" + 0.028*"cancel" + 0.023*"plan" + 0.022*"affect" + 0.020*"march" + 0.019*"month" + 0.019*"infect" + 0.018*"stay" + 0.018*"trip"
Topic: 4 
Words: 0.108*"work" + 0.061*"home" + 0.022*"time" + 0.019*"covid" + 0.017*"care" + 0.017*"peopl" + 0.015*"reduc" + 0.015*"hour" + 0.014*"famili" + 0.014*"school"
Topic: 5 
Words: 0.103*"work" + 0.065*"home" + 0.023*"groceri" + 0.021*"school" + 0.017*"shop" + 0.017*"quarantin" + 0.016*"close" +

In [213]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.101*"home" + 0.088*"work" + 0.044*"stay" + 0.026*"affect" + '
  '0.025*"family" + 0.025*"income" + 0.024*"time" + 0.021*"limit" + '
  '0.019*"store" + 0.016*"reduce"'),
 (1,
  '0.072*"work" + 0.050*"home" + 0.034*"affect" + 0.031*"family" + '
  '0.021*"time" + 0.017*"virus" + 0.015*"health" + 0.015*"need" + '
  '0.014*"school" + 0.014*"covid"'),
 (2,
  '0.117*"work" + 0.059*"home" + 0.029*"company" + 0.024*"income" + '
  '0.022*"time" + 0.022*"friends" + 0.021*"furlough" + 0.020*"online" + '
  '0.019*"cancel" + 0.019*"family"'),
 (3,
  '0.075*"work" + 0.064*"home" + 0.054*"time" + 0.042*"cause" + 0.032*"lose" + '
  '0.023*"money" + 0.021*"worry" + 0.019*"stress" + 0.017*"situation" + '
  '0.017*"school"'),
 (4,
  '0.066*"affect" + 0.042*"work" + 0.041*"coronavirus" + 0.035*"home" + '
  '0.034*"people" + 0.029*"hand" + 0.028*"virus" + 0.021*"fear" + '
  '0.019*"weeks" + 0.018*"life"'),
 (5,
  '0.114*"work" + 0.088*"home" + 0.034*"close" + 0.030*"school" + 0.029*"stay" '
  '+ 0

In [144]:
!pip install pyLDAvis==2.1.2

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=658f587df7d592ba603d5ee1cfe68e7ad834fc581e7539d168701de050f27ffc
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\59\70\ee\99a0df99d8b4a7b87c79640ebee0927c0f6ccff046e9cc2471
Successfully built pyLDAvis
  Attempting uninstall: pyLDAvis
    Found existing installation: pyLDAvis 3.3.1
    Uninstalling pyLDAvis-3.3.1:
      Successfully uninstalled pyLDAvis-3.3.1


# pyLDAvis

In [215]:
#https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

import os # new
from pathlib import Path

import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()

#df = pd.read_csv("Sanctions/chord/housing.csv")

LDAvis_data_filepath = os.path.join('Sanctions/chord/results/CV2_LDAvis_prepared0_'+str(num_topics))# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, 'Sanctions/chord/results/CV2_ldavis_prepared0_'+ str(num_topics) +'.html')
    LDAvis_prepared

D:\Anaconda\Anaconda_installation\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  term_ix = topic_terms.unique()


# Running LDA using TF-IDF

In [242]:

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)


In [243]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))



Topic: 0 Word: 0.041*"quarantin" + 0.029*"home" + 0.028*"furlough" + 0.027*"lose" + 0.023*"work" + 0.023*"abl" + 0.021*"time" + 0.021*"leav" + 0.018*"month" + 0.017*"lockdown"
Topic: 1 Word: 0.020*"peopl" + 0.019*"time" + 0.018*"work" + 0.016*"depress" + 0.015*"health" + 0.015*"worri" + 0.015*"lack" + 0.014*"covid" + 0.013*"stick" + 0.013*"week"
Topic: 2 Word: 0.036*"home" + 0.034*"work" + 0.023*"need" + 0.023*"time" + 0.023*"school" + 0.020*"store" + 0.018*"go" + 0.017*"kid" + 0.017*"groceri" + 0.016*"social"
Topic: 3 Word: 0.039*"coronavirus" + 0.024*"econom" + 0.020*"affect" + 0.019*"leav" + 0.019*"forc" + 0.019*"famili" + 0.018*"hous" + 0.018*"work" + 0.018*"person" + 0.018*"caus"
Topic: 4 Word: 0.047*"incom" + 0.026*"affect" + 0.022*"financ" + 0.021*"work" + 0.020*"home" + 0.019*"quarantin" + 0.018*"life" + 0.018*"covid" + 0.017*"isol" + 0.016*"hand"
Topic: 5 Word: 0.053*"work" + 0.049*"affect" + 0.022*"close" + 0.021*"shut" + 0.020*"home" + 0.020*"countri" + 0.016*"univers" + 0.0

# Classification of the topics
## Performance evaluation by classifying sample document using LDA Bag of Words model

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

In [ ]:
processed_docs[600]

In [ ]:
for index, score in sorted(lda_model[bow_corpus[600]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


In [112]:
documents['CV2'][600]

'Mostly the only real change is that I am working from home and not seeing anyone in person -- I am taking more calls and reaching out to friends via zoom for socialization, and going to "virtual" events at times. I also am making an effort to do things that "fill me up", like hobbies and exercise and reading. I am cooking all my food at home, and freezing leftovers and produce to avoid having to go to the stores, which are out of many items.'

In [115]:
documents['CV2'][1]

'I could not find a job now because everything is closed. I am a student and university is also closed. Just at home until the shut down is over.'

In [110]:
processed_docs[1]

['close', 'student', 'univers', 'close', 'home', 'shut']

In [111]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8713736534118652	 
Topic: 0.072*"home" + 0.053*"work" + 0.042*"famili" + 0.041*"incom" + 0.033*"close" + 0.027*"school" + 0.025*"quarantin" + 0.023*"reduc" + 0.022*"stress" + 0.021*"econom"

Score: 0.014303410425782204	 
Topic: 0.073*"home" + 0.062*"work" + 0.047*"affect" + 0.034*"limit" + 0.026*"stay" + 0.026*"life" + 0.022*"covid" + 0.020*"social" + 0.020*"food" + 0.020*"peopl"

Score: 0.01429315097630024	 
Topic: 0.115*"work" + 0.063*"home" + 0.022*"time" + 0.018*"care" + 0.018*"covid" + 0.015*"hour" + 0.015*"school" + 0.015*"peopl" + 0.014*"month" + 0.014*"famili"

Score: 0.014292443171143532	 
Topic: 0.098*"work" + 0.063*"home" + 0.023*"groceri" + 0.020*"school" + 0.019*"time" + 0.018*"shop" + 0.017*"quarantin" + 0.015*"close" + 0.015*"week" + 0.015*"money"

Score: 0.014292404986917973	 
Topic: 0.083*"work" + 0.051*"home" + 0.047*"affect" + 0.022*"go" + 0.018*"health" + 0.017*"school" + 0.016*"wife" + 0.016*"life" + 0.016*"covid" + 0.015*"social"

Score: 0.01429217960685

# Performance evaluation by classifying sample document using LDA TF-IDF model

In [78]:
for index, score in sorted(lda_model_tfidf[bow_corpus[200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))




Score: 0.7749688029289246	 
Topic: 0.066*"work" + 0.051*"home" + 0.023*"famili" + 0.022*"time" + 0.021*"social" + 0.019*"school" + 0.018*"abl" + 0.017*"quarantin" + 0.015*"month" + 0.015*"close"

Score: 0.025008346885442734	 
Topic: 0.034*"life" + 0.028*"caus" + 0.025*"covid" + 0.022*"affect" + 0.020*"financ" + 0.019*"money" + 0.018*"furlough" + 0.016*"person" + 0.016*"work" + 0.015*"place"

Score: 0.025004662573337555	 
Topic: 0.038*"stay" + 0.034*"home" + 0.030*"feel" + 0.023*"work" + 0.022*"time" + 0.021*"current" + 0.019*"week" + 0.016*"stick" + 0.016*"school" + 0.016*"anxieti"

Score: 0.025003833696246147	 
Topic: 0.019*"get" + 0.018*"worri" + 0.018*"hous" + 0.018*"work" + 0.018*"outsid" + 0.016*"coronavirus" + 0.016*"affect" + 0.016*"peopl" + 0.015*"go" + 0.015*"famili"

Score: 0.02500378154218197	 
Topic: 0.032*"home" + 0.028*"work" + 0.024*"stay" + 0.020*"friend" + 0.018*"groceri" + 0.017*"need" + 0.017*"famili" + 0.016*"relat" + 0.016*"peopl" + 0.016*"affect"

Score: 0.025003

# Testing model on unseen document

In [80]:
unseen_document = """I strictly adhere to health regulations. I sleep less at night because I'm afraid that it will happen to me or one of my family members. I am much more irritable and nervous. I started training at home. I haven't been out for about 35 days. Covid 19 has stopped the renovations in our new home. I don't have free time because my children are small and the kindergartens are closed."""
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.7162970900535583	 Topic: 0.072*"home" + 0.053*"work" + 0.042*"famili" + 0.041*"incom" + 0.033*"close"
Score: 0.23035505414009094	 Topic: 0.098*"work" + 0.063*"home" + 0.023*"groceri" + 0.020*"school" + 0.019*"time"


# CV2 column LDAvis topic visualization

LDAviz tool helps visualization of the multi-dimensional data into 2 dimensions.



In [ ]:
#https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

import os # new
from pathlib import Path

import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()

#df = pd.read_csv("Sanctions/chord/housing.csv")

LDAvis_data_filepath = os.path.join('Sanctions/chord/results/LDAvis_prepared_'+str(num_topics))# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, 'Sanctions/chord/results/ldavis_prepared_'+ str(num_topics) +'.html')
    LDAvis_prepared

# Reading the LDAvis interactive topic-modeling data visualization 

- The size of the Circle on the left describes the importance of that topic in the model/corpus.
- You can hover over the circle representing one specific topic on the left to see the words change on the graph to the right related to the specific topic circle you selected.
- When you highlight the words on the right (eg: home, stay, etc.,) in the graph, the same sort of areas overall of the topic space gets highlighted.

# Limitation of LDAVis

- Its really difficult to determine the relative sizes of the circles.(like areas the areas of the circles 2 times, 5 times or 10 times bigger/smaller than each other).
- The X and Y axis do not matter. They do not give any information and are meaningless. Note that LDAvis is a projection of the multi-dimensional space into two dimensions
- The size and location of the circles are not related to each other.

# SANC2 data

In [169]:
# reading data from csv
df = pd.read_csv("Sanctions/chord/highlighted_sentiments_survey.csv")
df['SANC2'][925] # before 925 are empty (NULL)

'Severe inflation has ruined our lives'

In [180]:
# Remove punctuation
df['SANC2_text_processed'] = \
df['SANC2'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase
df['SANC2_text_processed'] = \
df['SANC2_text_processed'].map(lambda x: x.lower())# Print out the first rows of papers
df['SANC2_text_processed'].tail()

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
C:\Users\User\AppData\Local\Temp/ipykernel_20668/2574139200.py:3: DeprecationWarning: invalid escape sequence \.
  df['SANC2'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase


1145    lack of availability of modern technologies\nl...
1146    rising prices of consumer goods that have impo...
1147    impossibility of foreign marketing weakness in...
1148                       sanctions reduce opportunities
1149    lack of access to health and pharmaceutical ma...
Name: SANC2_text_processed, dtype: object

In [181]:
df['SANC2'].fillna('').astype(str)

0                                                  #NULL!
1                                                  #NULL!
2                                                  #NULL!
3                                                  #NULL!
4                                                  #NULL!
                              ...                        
1145    Lack of availability of modern technologies\nL...
1146    Rising prices of consumer goods that have impo...
1147    Impossibility of foreign marketing, weakness i...
1148                       Sanctions reduce opportunities
1149    Lack of access to health and pharmaceutical ma...
Name: SANC2, Length: 1150, dtype: object

In [182]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_words)# Create Corpus
texts = data_words# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]# View
print(corpus[:1][0][:30])



[(0, 1)]


In [183]:
from pprint import pprint# number of topics
num_topics = 10# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"sanctions" + 0.017*"economic" + 0.014*"products" + 0.014*"decrease" '
  '+ 0.013*"null" + 0.011*"affected" + 0.010*"impact" + 0.010*"lives" + '
  '0.010*"food" + 0.008*"work"'),
 (1,
  '0.021*"sanctions" + 0.021*"purchasing" + 0.021*"power" + 0.017*"drug" + '
  '0.016*"decrease" + 0.013*"lack" + 0.010*"economic" + 0.010*"null" + '
  '0.010*"reduction" + 0.010*"difficulty"'),
 (2,
  '0.348*"null" + 0.027*"sanctions" + 0.011*"increase" + 0.010*"financial" + '
  '0.008*"prices" + 0.008*"price" + 0.008*"exchange" + 0.007*"economic" + '
  '0.007*"rising" + 0.006*"expenses"'),
 (3,
  '0.024*"goods" + 0.020*"economic" + 0.019*"sanctions" + 0.011*"lack" + '
  '0.010*"problems" + 0.010*"countries" + 0.010*"expensive" + 0.009*"people" + '
  '0.009*"impossibility" + 0.009*"raw"'),
 (4,
  '0.210*"null" + 0.014*"sanctions" + 0.013*"impact" + 0.012*"goods" + '
  '0.011*"international" + 0.010*"prices" + 0.010*"foreign" + 0.010*"biggest" '
  '+ 0.009*"inflation" + 0.009*"companies"'),


In [184]:
import pyLDAvis.gensim_models as gensimvis
#https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
pyLDAvis.enable_notebook()

import os # new
from pathlib import Path

import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()

#df = pd.read_csv("Sanctions/chord/housing.csv")

LDAvis_data_filepath = os.path.join('Sanctions/chord/SANC2_topics/SANC2_LDAvis_prepared_0'+str(num_topics))# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, 'Sanctions/chord/SANC2_topics/SANC2_LDAvis_prepared_0'+ str(num_topics) +'.html')
    LDAvis_prepared

# CV2 without stemming   using 2nd method like SANC2 topic modeling (gives error)

In [192]:
# reading data from csv
df = pd.read_csv("Sanctions/chord/highlighted_sentiments_survey.csv")
df['CV2'][925] # before 925 are empty (NULL)

'It may make it harder to sign a new contract unless a direct or indirect service is provided, otherwise the rest of the time will be marginalized for a long time.'

In [189]:
#df['CV2'].astype(str)

0       Store i was working in had to close due the re...
1       I could not find a job now because everything ...
2       Lectures are cancelled and exams postponed, it...
3       During the last month I am not working in a bi...
4       I have not been able to go to work which made ...
                              ...                        
1145    It had positive effects\nEveryone was paying a...
1146    Lack of simple entertainment and gathering tha...
1147                         Almost the same as the above
1148                                       Overall useful
1149    Closing the kindergarten and as a result quitt...
Name: CV2, Length: 1150, dtype: object

In [190]:
#df['CV2'].fillna('').astype(str)

0       Store i was working in had to close due the re...
1       I could not find a job now because everything ...
2       Lectures are cancelled and exams postponed, it...
3       During the last month I am not working in a bi...
4       I have not been able to go to work which made ...
                              ...                        
1145    It had positive effects\nEveryone was paying a...
1146    Lack of simple entertainment and gathering tha...
1147                         Almost the same as the above
1148                                       Overall useful
1149    Closing the kindergarten and as a result quitt...
Name: CV2, Length: 1150, dtype: object

In [193]:
# Remove punctuation
df['CV2_text_processed'] = \
df['CV2'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase
df['CV2_text_processed'] = \
df['CV2_text_processed'].map(lambda x: x.lower())# Print out the first rows of papers
df['CV2_text_processed'].tail()

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
C:\Users\User\AppData\Local\Temp/ipykernel_20668/4190666080.py:3: DeprecationWarning: invalid escape sequence \.
  df['CV2'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase


TypeError: expected string or bytes-like object

In [ ]:
#df['CV2'].fillna('').astype(str)

In [ ]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_words)# Create Corpus
texts = data_words# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]# View
print(corpus[:1][0][:30])


In [ ]:
from pprint import pprint# number of topics
num_topics = 10# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis.gensim_models as gensimvis
#https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
pyLDAvis.enable_notebook()

import os # new
from pathlib import Path

import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()

#df = pd.read_csv("Sanctions/chord/housing.csv")

LDAvis_data_filepath = os.path.join('Sanctions/chord/results/CV2_topics/CV2_LDAvis_prepared_000'+str(num_topics))# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, 'Sanctions/chord/results/CV2_topics/CV2_LDAvis_prepared_000'+ str(num_topics) +'.html')
    LDAvis_prepared